## RockPaperScissorsLizardSpock.js

In this notebook we will finetune a SqueezeNet pretrained on ImageNet to classify between the classes 'rock', 'paper', 'scissors', 'lizard', 'spock' and 'other'.

### Data setup

There should already be a folder for each class in `./data/train/`. E.g. all `rock` images should be in `./data/train/rock`

In [1]:
import os
import numpy as np

In [2]:
data_dir = './data/'
model_dir = data_dir + 'models/'
train_dir = data_dir + 'train/'
val_dir = data_dir + 'val/'

The output of the next cell should be (in any order) `['lizard', 'spock', 'other', 'paper', 'scissors', 'rock']`

In [3]:
classes = os.listdir(train_dir)
num_classes = len(classes)
classes

['lizard', 'spock', 'other', 'paper', 'scissors', 'rock']

In [4]:
os.makedirs(model_dir, exist_ok=True)

Create a train/validation split. 

In [5]:
os.makedirs(val_dir, exist_ok=True)

In [6]:
for c in classes:
    os.makedirs(val_dir + c, exist_ok=True)

In [7]:
def make_val_split(p):
    """
    Move random images of each category from train to val.
    p is the percentage of images to move, e.g.
    p == 0.2 will move 20% of each category to val.
    """
    for c in classes:
        file_names = os.listdir(train_dir + c)
        permutation = np.random.permutation(file_names)
        for i in range(int(len(file_names)*p)):
            file_path = os.path.join(c, permutation[i])
            os.rename(os.path.join(train_dir, file_path), 
                      os.path.join(val_dir, file_path))

In [8]:
# only do this once
make_val_split(0.2)

### Create and train the model

In [9]:
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, Dropout, Activation, GlobalAveragePooling2D
from keras.models import Model

Using TensorFlow backend.


We will use SqueezeNet but replace its top layers (the classification layers) to classify between 'rock', 'paper', 'scissors', 'lizard', 'spock' and 'other' and not between the 1000 ImageNet classes.

In [10]:
def get_model():
    base_model = SqueezeNet(weights='imagenet', include_top=False, input_shape=(227,227,3))
    x = base_model.output
    x = Dropout(0.5, name='drop9')(x)
    x = Convolution2D(num_classes, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)

    predictions = Activation('softmax', name='predictions')(x)

    return Model(inputs=base_model.input, outputs=predictions)

In [11]:
model = get_model()

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 113, 113, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 113, 113, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 56, 56, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [13]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [14]:
batch_size = 128

In [15]:
def get_batches(dirname, generator=ImageDataGenerator(), shuffle=True,
                batch_size=batch_size, class_mode='categorical', target_size=(227,227)):
    return generator.flow_from_directory(dirname, shuffle=shuffle, batch_size=batch_size,
                                 class_mode=class_mode, target_size=target_size)

The ImageDataGenerator will subtract the imagenet mean from each image and do some data augmentation.

In [16]:
def subtract_imagenet_mean(x):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

In [17]:
train_gen = ImageDataGenerator(preprocessing_function=subtract_imagenet_mean,
                                rotation_range=15, 
                                height_shift_range=0.1, 
                                shear_range=0.1,
                                zoom_range=0.1,
                                width_shift_range=0.1)
train_batches = get_batches(train_dir, generator=train_gen)

val_gen = ImageDataGenerator(preprocessing_function=subtract_imagenet_mean)
val_batches = get_batches(val_dir, generator=val_gen)

Found 14177 images belonging to 6 classes.
Found 3540 images belonging to 6 classes.


In [18]:
num_train = train_batches.samples
num_val = val_batches.samples

In [19]:
def fit(model, epochs):
    model.fit_generator(train_batches, 
                        steps_per_epoch=num_train // batch_size,
                        epochs=epochs,
                        validation_data=val_batches,
                        validation_steps=num_val // batch_size)

In [20]:
model.optimizer.lr = 0.0001
fit(model, 5)

Epoch 1/5
110/110 [==============================] - 121s - loss: 1.4623 - acc: 0.4243 - val_loss: 0.7275 - val_acc: 0.7428
Epoch 2/5
110/110 [==============================] - 112s - loss: 0.5796 - acc: 0.7931 - val_loss: 0.2845 - val_acc: 0.9051
Epoch 3/5
110/110 [==============================] - 111s - loss: 0.2812 - acc: 0.9047 - val_loss: 0.1186 - val_acc: 0.9618
Epoch 4/5
110/110 [==============================] - 111s - loss: 0.1971 - acc: 0.9349 - val_loss: 0.0925 - val_acc: 0.9714
Epoch 5/5
110/110 [==============================] - 111s - loss: 0.1276 - acc: 0.9574 - val_loss: 0.0526 - val_acc: 0.9826


In [22]:
fit(model, 5)

Epoch 1/5
110/110 [==============================] - 121s - loss: 0.0934 - acc: 0.9701 - val_loss: 0.0675 - val_acc: 0.9792
Epoch 2/5
110/110 [==============================] - 111s - loss: 0.0812 - acc: 0.9751 - val_loss: 0.0310 - val_acc: 0.9902
Epoch 3/5
110/110 [==============================] - 111s - loss: 0.0718 - acc: 0.9762 - val_loss: 0.0552 - val_acc: 0.9841
Epoch 4/5
110/110 [==============================] - 111s - loss: 0.0588 - acc: 0.9813 - val_loss: 0.0141 - val_acc: 0.9954
Epoch 5/5
110/110 [==============================] - 111s - loss: 0.0463 - acc: 0.9853 - val_loss: 0.0137 - val_acc: 0.9954


In [23]:
model.optimizer.lr = 1e-5
fit(model, 5)

Epoch 1/5
110/110 [==============================] - 120s - loss: 0.0489 - acc: 0.9837 - val_loss: 0.0220 - val_acc: 0.9907
Epoch 2/5
110/110 [==============================] - 111s - loss: 0.0461 - acc: 0.9851 - val_loss: 0.0076 - val_acc: 0.9983
Epoch 3/5
110/110 [==============================] - 111s - loss: 0.0329 - acc: 0.9898 - val_loss: 0.0049 - val_acc: 0.9980
Epoch 4/5
110/110 [==============================] - 111s - loss: 0.0297 - acc: 0.9907 - val_loss: 0.0066 - val_acc: 0.9977
Epoch 5/5
110/110 [==============================] - 111s - loss: 0.0332 - acc: 0.9893 - val_loss: 0.0078 - val_acc: 0.9977


In [25]:
model.optimizer.lr = 1e-6
fit(model, 5)

Epoch 1/5
110/110 [==============================] - 120s - loss: 0.0120 - acc: 0.9965 - val_loss: 0.0041 - val_acc: 0.9980
Epoch 2/5
110/110 [==============================] - 111s - loss: 0.0182 - acc: 0.9946 - val_loss: 0.0046 - val_acc: 0.9991
Epoch 3/5
110/110 [==============================] - 111s - loss: 0.0137 - acc: 0.9960 - val_loss: 0.0076 - val_acc: 0.9962
Epoch 4/5
110/110 [==============================] - 111s - loss: 0.0133 - acc: 0.9952 - val_loss: 0.0055 - val_acc: 0.9991
Epoch 5/5
110/110 [==============================] - 111s - loss: 0.0202 - acc: 0.9940 - val_loss: 0.0050 - val_acc: 0.9977


In [26]:
model.optimizer.lr = 1e-7
fit(model, 5)

Epoch 1/5
110/110 [==============================] - 120s - loss: 0.0098 - acc: 0.9966 - val_loss: 0.0022 - val_acc: 0.9991
Epoch 2/5
110/110 [==============================] - 111s - loss: 0.0164 - acc: 0.9951 - val_loss: 0.0050 - val_acc: 0.9983
Epoch 3/5
110/110 [==============================] - 111s - loss: 0.0110 - acc: 0.9961 - val_loss: 0.0030 - val_acc: 0.9988
Epoch 4/5
110/110 [==============================] - 111s - loss: 0.0084 - acc: 0.9969 - val_loss: 0.0033 - val_acc: 0.9991
Epoch 5/5
110/110 [==============================] - 111s - loss: 0.0179 - acc: 0.9943 - val_loss: 0.0033 - val_acc: 0.9986


Save the weights for the case they are needed again.

In [28]:
model.save_weights(data_dir + 'models/keras-weights.hdf5')

To load them do
```python
model.load_weights(data_dir + 'models/keras-weights.hdf5')
```

### Export model and weights for deeplearn.js

In [29]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model

The deeplearn.js repository provides the script `dump_checkpoint_vars.py` which takes a tensorflow checkpoint and converts it for use with deeplearn.js. To use this, we will first save the model as a tensorflow checkpoint.

In deeplearn.js this model will only be used for inference, no training will be done. So we have to tell keras to use the model in 'inference mode' where some layers behave differently (e.g. Dropout).

In [30]:
K.set_learning_phase(0)

The model has to be rebuilt for this to take effect:

In [31]:
config = model.get_config()
weights = model.get_weights()

model = Model.from_config(config)
model.set_weights(weights)

Save the checkpoint:

In [32]:
sess = K.get_session()
saver = tf.train.Saver()
saver.save(sess, model_dir + 'model.ckpt')

'./data/models/model.ckpt'

Use a slightly modified version of `dump_checkpoint_vars.py` from deeplearn.js to convert the checkpoint:

In [33]:
%run dump_checkpoint_vars.py --output_dir=deeplearn-checkpoint \
--checkpoint_file=data/models/model.ckpt \
--remove_variables_regex training*|Adam*

Writing variable fire6/expand3x3_1/bias as fire6/expand3x3/bias
Writing variable fire6/expand3x3_1/kernel as fire6/expand3x3/kernel
Ignoring Adam/beta_1
Ignoring Adam/beta_2
Writing variable fire7/expand1x1_1/bias as fire7/expand1x1/bias
Writing variable fire7/expand1x1_1/kernel as fire7/expand1x1/kernel
Writing variable fire7/squeeze1x1_1/bias as fire7/squeeze1x1/bias
Writing variable fire7/squeeze1x1_1/kernel as fire7/squeeze1x1/kernel
Writing variable fire9/expand1x1_1/bias as fire9/expand1x1/bias
Writing variable fire9/expand1x1_1/kernel as fire9/expand1x1/kernel
Ignoring training/Adam/Variable_98
Ignoring training/Adam/Variable_99
Writing variable fire6/squeeze1x1_1/bias as fire6/squeeze1x1/bias
Writing variable fire6/squeeze1x1_1/kernel as fire6/squeeze1x1/kernel
Writing variable fire2/expand3x3_1/bias as fire2/expand3x3/bias
Writing variable fire2/expand3x3_1/kernel as fire2/expand3x3/kernel
Writing variable fire8/squeeze1x1_1/bias as fire8/squeeze1x1/bias
Writing variable fire8

This should have created the folder `deeplearn-checkpoint` in the current directory.